In [34]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
import tavily

In [35]:
from langgraph.checkpoint.memory import InMemorySaver

In [36]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [37]:

OPEN_API_KEY= os.getenv("OPEN_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o-mini",     
    temperature=0.0,
    api_key=OPEN_API_KEY
)

In [38]:
from tavily import TavilyClient
client = TavilyClient(os.getenv("TAVILY_API_KEY"))
response = client.search(
    query="delhi red fort blast"
)
print(response)

{'query': 'delhi red fort blast', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.aljazeera.com/news/liveblog/2025/11/11/delhi-red-fort-blast-live-terrorism-law-invoked-in-india-after-9-killed', 'title': "Delhi Red Fort blast updates: India's Modi alleges 'conspiracy' as 13 ...", 'content': 'Skip links[Skip to Content](https://www.aljazeera.com/news/liveblog/2025/11/11/delhi-red-fort-blast-live-terrorism-law-invoked-in-india-after-9-killed#main-content-area) Sign up[Enrich your Al Jazeera experience by signing in or creating an account.](https://www.aljazeera.com/news/liveblog/2025/11/11/delhi-red-fort-blast-live-terrorism-law-invoked-in-india-after-9-killed) Sign up[Enrich your Al Jazeera experience by signing in or creating an account.](https://www.aljazeera.com/news/liveblog/2025/11/11/delhi-red-fort-blast-live-terrorism-law-invoked-in-india-after-9-killed) [](https://www.facebook.com/sharer/sharer.php?u=https%3A%2F%2Faje.io%2Fherov6)[](ht

In [39]:
from typing import TypedDict, Annotated, List, Dict, Any

In [40]:
import operator

In [41]:
from typing import TypedDict

class AgentState(TypedDict):
    question: str
    history: Annotated[List[Dict[str, str]], operator.add]
    search_results: dict  
    answer: str

In [42]:
# def rewrite_query(state: AgentState):
#     """
#     Node 0: Query Rewriting

#     Refines the user's initial question to be more specific and context-aware,
#     improving the quality of subsequent search results.
#     """
#     user_question = (state.get("question") or "").strip()
#     history = state.get("history", []) or []

#     # Build a tiny context from last few turns (if present)
#     context = ""
#     if history:
#         context = "\n".join(f"{m.get('role','user')}: {m.get('content','')}" for m in history[-6:])

#     system_prompt = (
#         "You rewrite a user's message into ONE precise web-search query.\n"
#         "Rules:\n"
#         "- Stay strictly on the same incident/topic suggested by the provided context/history.\n"
#         "- Include WHERE (city/country) and WHEN (specific date window) if present.\n"
#         "- Use 3–8 essential keywords; remove filler words.\n"
#         "- If the prompt is vague (e.g., 'government reaction on this'), assume it's about the SAME incident.\n"
#         "- Prefer neutral wording, no quotes, no extra text. Output ONLY the final query string."
#     )

#     user_prompt = f"""[CONTEXT]
# {context}

# [USER MESSAGE]
# {user_question}

# [INSTRUCTION]
# Rewrite into ONE precise web search query for this SAME incident/date/location.
# Return ONLY the query string, nothing else.
# """.strip()

#     try:
#         # Invoke with messages and take the content string
#         msg = llm.invoke([
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": user_prompt},
#         ])
#         rewritten = (getattr(msg, "content", "") or "").strip()

#         # Strip accidental surrounding quotes/backticks
#         if len(rewritten) > 1 and rewritten[0] in ("'", '"', "`") and rewritten[-1] == rewritten[0]:
#             rewritten = rewritten[1:-1].strip()

#         return {"question": rewritten or user_question}
#     except Exception:
#         # Safe fallback
#         return {"question": user_question}


In [43]:
# rewrite_query.py (only the body of the function matters)
from langchain_core.messages import SystemMessage, HumanMessage

def rewrite_query(state: AgentState):
    user_question = (state.get("question") or "").strip()
    history = state.get("history", []) or []

    context = "\n".join(f"{m.get('role','user')}: {m.get('content','')}" for m in history[-6:])

    system_prompt = (
        "You rewrite a user's message into ONE precise web-search query.\n"
        "Rules:\n"
        "- Stay strictly on the same incident/topic suggested by the provided context/history.\n"
        "- Include WHERE and WHEN if present.\n"
        "- Use 3–8 essential keywords; remove filler.\n"
        "- If the prompt is vague (e.g., 'government reaction on this'), assume SAME incident.\n"
        "- Output ONLY the final query string (no quotes)."
    )
    user_prompt = f"""[CONTEXT]
{context}

[USER MESSAGE]
{user_question}

[INSTRUCTION]
Rewrite into ONE precise on-topic web search query for the SAME incident/date/location.
Only the query string."""
    msg = llm.invoke([SystemMessage(content=system_prompt),
                      HumanMessage(content=user_prompt)])
    rewritten = (getattr(msg, "content", "") or "").strip()
    if len(rewritten) > 1 and rewritten[0] in ("'", '"', "`") and rewritten[-1] == rewritten[0]:
        rewritten = rewritten[1:-1].strip()
    return {"question": rewritten or user_question}


In [44]:
def search_web(state: AgentState):
    """
    Node 1: Intelligent Web Search

    Uses Tavily's AI-optimized search to find and process web information.
    Behind the scenes: Tavily searches multiple sources, extracts relevant content,
    and uses AI to synthesize the information into a coherent answer.
    """
    #print(f"🔍 Searching: {state['question']}")
    client = TavilyClient(os.getenv("TAVILY_API_KEY"))


    search_results = client.search(
        query=state["question"],
        max_results=3,           # Number of sources to aggregate
        include_answer=True      # Get AI-generated answer, not just links!
    )

    return {"search_results": search_results}

In [45]:
def generate_answer(state: AgentState):
    """
    Node 2: Answer Synthesis and Formatting

    Takes the search results from Tavily and formats them into a clean,
    user-friendly response with proper source attribution.
    """
    #print("🤖 Formatting answer...")

    # Extract Tavily's AI-generated answer (the smart synthesis)
    ai_answer = state["search_results"].get("answer", "No answer found")

    # Extract source URLs for transparency and verification
    sources = [f"- {result['title']}: {result['url']}" 
              for result in state["search_results"]["results"]]

    # Combine the intelligent answer with source attribution
    final_answer = f"{ai_answer}\n\nSources:\n" + "\n".join(sources)

    return {"answer": final_answer}
   
    

In [46]:
from typing import List
from pydantic import BaseModel

In [47]:
class NewsSummary(BaseModel):
    summary: str
    sources: List[str]


In [48]:
def summarize_answer(state: AgentState):
    """
    Node 3: Final Summary Refinement

    Further refines the generated answer for clarity, conciseness, and readability.
    """
    formatted_answer = state.get("answer", "").strip()
    results = (state.get("search_results") or {}).get("results", []) or []

    docs = []
    for r in results[:5]:
        title = r.get("title", "Untitled")
        url = r.get("url", "")
        content = r.get("content") or r.get("snippet") or ""
        docs.append(f"Title: {title}\nURL: {url}\nContent:\n{content}")
    corpus = "\n\n---\n\n".join(docs)

    prompt = f"""
You are a factual news summarizer. Write a concise, neutral, 6–10 sentence summary
for the user's question below using the provided source extracts AND the formatted answer.
Lead with what/when/where (use concrete dates if present), include key developments and
any official statements, avoid speculation, and flag disputed points. End with a short
'Sources:' list of only links of sources.

User question: {state.get('question', '')}

Formatted answer (for context):
{formatted_answer}

Source extracts:
{corpus}
"""
    structred_llm=llm.with_structured_output(NewsSummary.model_json_schema())
    summary = structred_llm.invoke(prompt)
    history = state.get("history", []) or []
    history = history + [{"role": "assistant", "content": summary}]

    #return {"answer": summary}
    return {"answer": summary, "history": history}


In [49]:
def create_agent():
    """
    Build the AI Agent Workflow

    This creates a StateGraph where:
    - Each node is an independent function that can read/update shared state
    - LangGraph handles orchestration, state management, and execution flow
    - Easy to extend: just add more nodes and define their connections
    """
    # Create the workflow graph
    workflow = StateGraph(AgentState)

    # Define our processing nodes
    workflow.add_node("rewrite", rewrite_query)   # Step 0: Refine the query
    workflow.add_node("search", search_web)        # Step 1: Gather information  
    workflow.add_node("result", generate_answer)   # Step 2: Process and format
    workflow.add_node("summarize", summarize_answer)  # Step 3: Final refinement

    # Define the flow of intelligence
    workflow.set_entry_point("rewrite")
    workflow.add_edge("rewrite", "search")      # Start here
    workflow.add_edge("search", "result") 
    workflow.add_edge("result", "summarize")
    workflow.add_edge("summarize", END)
    #workflow.add_edge("result", END)        # After answer, we're done
    checkpointer = InMemorySaver()

    return workflow.compile(checkpointer=checkpointer)

In [50]:
agent = create_agent()

In [51]:
from langchain_core.runnables import RunnableConfig

In [52]:
config: RunnableConfig = {"configurable": {"thread_id": "14"}}
result=agent.invoke({"question": "delhi red fort blast"}, config=config)
print(result.get("answer", ""))

{'summary': 'On October 11, 2023, a car explosion near the historic Red Fort in New Delhi resulted in the deaths of at least eight people and injuries to several others. The blast occurred around 7 p.m. local time, close to a subway station, and caused significant damage to nearby vehicles. Authorities are treating the incident as a potential terrorist attack and have launched an investigation under anti-terrorism laws. Indian Prime Minister Narendra Modi stated that those responsible for the attack will be brought to justice. This incident marks the first major blast in the city in over a decade, prompting heightened security measures and forensic investigations at the site.', 'sources': ['https://www3.nhk.or.jp/nhkworld/en/news/20251111_04/', 'https://www.npr.org/2025/11/10/g-s1-97197/india-red-fort-explosion-new-delhi', 'https://www.arabnews.com/node/2622156/world']}


In [ ]:
result=agent.invoke({"question": "government reaction on this incident"}, config=config)
print(result.get("answer", ""))

{'summary': "On October 11, 2023, a car explosion near the Red Fort in New Delhi resulted in at least eight fatalities and 20 injuries. The explosion occurred close to a traffic signal near the Red Fort metro station, damaging several nearby vehicles. India's Home Minister Amit Shah confirmed that a Hyundai i20 was involved in the incident. The National Investigation Agency, India's anti-terrorism agency, is spearheading the investigation, with Prime Minister Narendra Modi promising to hold those responsible accountable. The exact cause of the explosion has not yet been determined, and police have not provided further details regarding the incident. The Red Fort is a significant historical site and the location of the Prime Minister's annual Independence Day address.", 'sources': ['https://www.cbsnews.com/news/new-delhi-india-car-explosion-red-fort-deaths-reported/', 'https://news3lv.com/news/nation-world/car-explosion-red-fort-india-kills-8-injures-20-investigation-delhi-narendra-modi